## Shell Contact Example

In [1]:
# !pip install debugpy

In [2]:
import numpy as np

import dismech


geom = dismech.GeomParams(rod_r0=0.005,
                          shell_h=0.005)

material = dismech.Material(density=1200,
                            youngs_rod=0,
                            youngs_shell=2e7,
                            poisson_rod=0,
                            poisson_shell=0.5)

shell_contact_sim = dismech.SimParams(static_sim=False,
                                  two_d_sim=False,   # no twisting
                                  use_mid_edge=False,
                                  use_line_search=False,
                                  show_floor=False,
                                  log_data=True,
                                  log_step=1,
                                  dt=1e-3,
                                  max_iter=120,
                                  total_time=1.0,
                                  plot_step=1,
                                  tol=1e-4,
                                  ftol=1e-4,
                                  dtol=1e-2)

env = dismech.Environment()
env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))
# env.add_force('floorContact', ground_z=-1e-3, stiffness=1e3, delta=2e-3, h=1e-3)
env.add_force('selfContact', delta=1e-4, h=5e-3, kc=1)
env.add_force('damping', eta = 20)
#env.set_static()   

geo = dismech.Geometry.from_txt('input_shell_more_curved.txt')
# geo = dismech.Geometry.from_txt('input_oneTriangle.txt')
# geo = dismech.Geometry.from_txt('input_twoTriangleContact_p2t.txt')

robot = dismech.SoftRobot(geom, material, geo, shell_contact_sim, env)

fixed_points = np.array(np.where(
    (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 0] <= 0.01) &
    (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 2] == 0.0)
)[0])

robot = robot.fix_nodes(fixed_points)

# u = robot.state.u
# u[11] = -0.5
# u[14] = -0.5
# u[17] = -0.5

### Time Stepping

As we are performing a static simulation, we must change gravity for each time step.

In [3]:
stepper = dismech.ImplicitEulerTimeStepper(robot)

robots = stepper.simulate()

qs = np.stack([robot.state.q for robot in robots])

delta: 0.02
h: 1.0
K1: 750.0
scale:  200.0
kc:  1
upper limit for quadratic: 1.98
upper limit for smooth: 2.02
158218.8706944033
374905.1820517846
244823.10503886684
471061.73192166013
318667.7925042584
1064294.8068487728
456514.8278296972
249884.5665422054
1103033.6142143533
427195.08653216896
808906.1640777913
282337.8434201264
225301.9888036427
193522.29197930908
201708.7690640993
226325.0679332625
451591.17919388047
284268.4998697397
218584.04838942626
228685.01501029142
284380.6389232518
218525.26786422948
203052.69365842998
202794.091857247
183360.87853220303
195007.66678300008
330936.23121542245
271437.7541577485
234598.13701884006
229649.5318356427
220636.51485831957
215342.2831612149
2607762.3264593137
2347338.4643089217
2113020.3929177714
1902158.3370044555
1711952.9688310875
1540953.182950026
1387098.9821200517
1248918.750806303
1125345.013962189
1012959.4654500562
913925.8041522353
822659.6858221887
741188.1521632129
669753.89387257
604411.0246518472
544830.9664937556
49749

ValueError: Iteration limit 120 reached before convergence

In [ ]:
t = np.arange(robot.sim_params.total_time, step=robot.sim_params.dt)
options = dismech.AnimationOptions(title='Shell contact p2p', plot_step=1)

fig = dismech.get_interactive_animation_plotly(robot, t, qs, options)
fig.show()

In [ ]:
#ani = dismech.get_animation(robot, t, qs, options)
#ani.save('5e-5.gif')